# Promethe Tri

This module shows an example of how to use the "Promethee Tri" module.

In [1]:
%config Completer.use_jedi = False

## Definition of inputs

In [2]:
import pandas as pd
import sys
sys.path.append('../..')

## Definition of inputs and problem formalization

In [3]:
from modular_parts.preference import compute_preference_indices
from modular_parts.flows import calculate_promethee_outranking_flows, calculate_net_outranking_flows
from modular_parts.sorting import calculate_prometheetri_sorted_alternatives
from core.enums import Direction, GeneralCriterion


alternatives = [f"a{i}" for i in range(1, 8)]
profiles = [f"p{i}" for i in range(1, 4)]
criteria = [f"c{i}" for i in range(1, 4)]
criteria_directions = pd.Series([Direction.MAX, Direction.MIN, Direction.MAX], index=criteria)
criteria_weights = pd.Series([0.3, 0.2, 0.5], index=criteria)
generalised_criteria = pd.Series([GeneralCriterion.U_SHAPE,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE,
                                  GeneralCriterion.USUAL], index=criteria)
preference_thresholds = pd.Series([2, 10, None], index=criteria)
indifference_thresholds = pd.Series([1, 5, None], index=criteria)
standard_deviations = pd.Series([None, None, None], index=criteria) # None, because we do not use GeneralCriterion.GAUSSIAN

alternatives_performances = pd.DataFrame([[15, 83, 21],
                                          [10, 90, 15],
                                          [11, 75, 20],
                                          [18, 59, 20],
                                          [17, 60, 28],
                                          [22, 44, 15],
                                          [13, 62, 22]], index=alternatives, columns=criteria)
profiles_performances = pd.DataFrame([[12, 80, 16],
                                      [15, 68, 18],
                                      [18, 55, 20]], index=profiles, columns=criteria)

categories = [f"C{i}" for i in range(1, 4)]

## Required preferences

In [4]:
_, partial_altenatives_vs_profiles_preferences = compute_preference_indices(alternatives_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights,
                                                                    profiles_performances)

_, partial_profiles_vs_profiles_preferences =compute_preference_indices(profiles_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights)

In [5]:
partial_altenatives_vs_profiles_preferences[0]

p1   p2  p3
criteria                
c1       a1   1  0.0   0
         a2   0  0.0   0
         a3   0  0.0   0
         a4   1  1.0   0
         a5   1  1.0   0
         a6   1  1.0   1
         a7   0  0.0   0
c2       a1   0  0.0   0
         a2   0  0.0   0
         a3   0  0.0   0
         a4   1  0.8   0
         a5   1  0.6   0
         a6   1  1.0   1
         a7   1  0.2   0
c3       a1   1  1.0   1
         a2   0  0.0   0
         a3   1  1.0   0
         a4   1  1.0   0
         a5   1  1.0   1
         a6   0  0.0   0
         a7   1  1.0   1

In [6]:
partial_altenatives_vs_profiles_preferences[1]

a1   a2   a3  a4  a5  a6   a7
criteria                                  
c1       p1   0  1.0  0.0   0   0   0  0.0
         p2   0  1.0  1.0   0   0   0  1.0
         p3   1  1.0  1.0   0   0   0  1.0
c2       p1   0  1.0  0.0   0   0   0  0.0
         p2   1  1.0  0.4   0   0   0  0.0
         p3   1  1.0  1.0   0   0   0  0.4
c3       p1   0  1.0  0.0   0   0   1  0.0
         p2   0  1.0  0.0   0   0   1  0.0
         p3   0  1.0  0.0   0   0   1  0.0

In [7]:
partial_profiles_vs_profiles_preferences

p1  p2  p3
criteria               
c1       p1   0   0   0
         p2   1   0   0
         p3   1   1   0
c2       p1   0   0   0
         p2   1   0   0
         p3   1   1   0
c3       p1   0   0   0
         p2   1   0   0
         p3   1   1   0

## Usage of Promethee Tri

In [8]:
precise_assignments = calculate_prometheetri_sorted_alternatives(categories,
                                                                 criteria_weights, 
                                                                 partial_altenatives_vs_profiles_preferences, 
                                                                 partial_profiles_vs_profiles_preferences,
                                                                 assign_to_better_class=True,
                                                                 use_marginal_value=True)
precise_assignments

a1    C2
a2    C1
a3    C2
a4    C3
a5    C3
a6    C1
a7    C3
dtype: object